In [1]:
import pandas as pd
import numpy as np

In [129]:
df_subgen = pd.read_csv("genero_subgenero.csv") #check
df_multi = pd.read_csv("multimedia.csv") #check
df_usuarios = pd.read_csv("usuarios.csv") #check
df_prove = pd.read_csv("proveedores.csv") #check
df_subs = pd.read_csv("subscripciones.csv")
df_vis = pd.read_csv("visualizaciones.csv")
df_pagos = pd.read_csv("pagos_v2.csv") 

In [263]:
# Tabla users se puede usar tal cual
df_subgen
df_gen = df_subgen["genero"].drop_duplicates()
df_subgenero = df_subgen[~df_subgen["nombre_subgenero"].isnull()]
df_gen = df_gen.reset_index()
df_subgenero
df_gen = df_gen["genero"].reset_index().rename({'index':'gid'}, axis=1)

#df_subgenero.join(df_gen, on="genero")
df_gen_subgen = df_gen.set_index('genero').join(df_subgen.set_index('genero'))
df_gen_subgen = df_gen_subgen.reset_index()[["gid","nombre_subgenero"]]
df_gen_subgen = df_gen_subgen[~df_gen_subgen["nombre_subgenero"].isnull()]
#df_gen_subgen.to_csv(folder + "genero_subgenero.csv", index=False)
#df_gen.to_csv("8)genero.csv", index=False)
df_gen_subgen = df_gen_subgen.rename(columns={"nombre_subgenero":"genero"})
df_gen_subgen = df_gen_subgen.merge(df_gen, on="genero", how="inner")
df_gen_subgen = df_gen_subgen.rename(columns={'gid_x': "gid" ,"gid_y":"subgid"})[["gid","subgid"]]
df_gen_subgen.to_csv("9)subgenero.csv", index=False)

In [233]:
# De la table df_prove se puede encontrar la info de 
df_prove2 = df_prove[["id", "nombre", "costo"]]
df_prove3 = df_prove2.drop_duplicates().sort_values(by="id")
df_prove3 #Esta tabla es equivalente a la que nosotros tenemos
folder = 'ToBDD/'
df_prove3.to_csv(folder + '6)proveedor.csv', index=False)
df_prove3

,id,nombre,costo
10,1,Netflix,4500
17,2,Amazon Prime Video,3700
26,3,Crunchyroll,4400
50,4,HBO+,3500
13,5,Funimation,3900
7,6,Disney+,6500
0,7,Star+,7100
11,8,Hulu,7000
3,9,Showtime,3200
184,10,Stan,7900


In [232]:
# Peliculas pagadas
df_pelipag = df_prove[~df_prove["precio"].isnull()].sort_values(by="id")[["id","costo", "pid","precio","disponibilidad"]]
df_pelipag
df_prove_serie = df_prove[~df_prove["sid"].isnull()][["id","sid"]].sort_values(by="id") #id_prove e id_serie
df_prove_pelifree = df_prove[~df_prove["pid"].isnull()]
df_peli_incluida = df_prove_pelifree[df_prove_pelifree["precio"].isnull()][["id", "pid"]].sort_values(by="id")
# id_prove, id_peli
#df_peli_incluida.to_csv(folder+'3)ofreceenplan.csv', index=False)
#df_prove_serie.to_csv(folder+'2.1)proveedor_serie.csv', index=False)
df_pelipag = df_pelipag.rename({'id':'id_prov'}, axis=1).sort_values(by="id_prov")
df_pelipag.to_csv(folder + '2)ofrece_pagadas.csv', index = False)
#df_peli_incluida

In [224]:
# Equivalente a columna la tabla de peliculas
df_peli = df_multi[["pid","titulo","puntuacion","clasificacion","duracion","año"]].drop_duplicates()
df_peli = df_peli[pd.notnull(df_peli["pid"])].sort_values(by="pid")
#df_peli.to_csv(folder + '1)peliculas.csv', index=False)
df_peli

,pid,titulo,puntuacion,clasificacion,duracion,año
241,1.0,Return to Babylon,7.1,G,75,2013
141,2.0,Life's a Beach,9.0,PG-13,100,2012
311,3.0,'77,2.1,G,113,2017
97,4.0,In My Sleep,1.6,PG,104,2010
90,5.0,The Evil Within,4.6,PG-13,98,2017
...,...,...,...,...,...,...
442,97.0,Waking Madison,5.2,G,89,2010
15,98.0,The Babymakers,6.6,NC-17,95,2012
57,99.0,Electric Slide,6.7,PG,95,2014
169,100.0,Krystal,2.0,PG-13,90,2017


In [220]:
df_serie = df_multi[["sid","cid","serie","titulo","puntuacion","año","clasificacion","genero","numero"]] 
df_serie = df_serie[pd.notnull(df_serie["sid"])].sort_values(by="sid")
df_serie_final = df_serie[["sid","serie","puntuacion","año","clasificacion","numero"]].drop_duplicates()\
                        .groupby(by=["serie","clasificacion"]).max("numero").reset_index()
df_serie_final = df_serie_final.rename(columns={'numero':'temporadas'})
df_serie_final = df_serie_final[["sid","serie","clasificacion","puntuacion","año","temporadas"]]
df_serie_final = df_serie_final.astype({"sid": int, "temporadas":int}, copy=True)
df_serie_final.to_csv(folder+'4)series.csv', index=False)
df_serie_final

,sid,serie,clasificacion,puntuacion,año,temporadas
0,11,13 reasons why,R,8.3,2017,3
1,6,31 minutos,G,10.0,2003,4
2,24,Ataque a los titanes,PG-13,9.8,2010,3
3,15,Atypical,PG-13,9.2,2017,3
4,26,Avatar: la leyenda de Aang,G,7.9,2005,3
5,5,Dark,PG-13,9.9,2017,3
6,30,Doctor Who,G,9.0,2005,2
7,16,Dr. House,R,9.3,2004,3
8,2,Friends,PG,9.5,1994,5
9,25,Full Metal Alchemist Brotherhood,PG,8.6,2003,3


In [239]:
df_capitulos = df_serie[["sid","cid","titulo","numero"]]
df_capitulos = df_capitulos.rename(columns={"titulo": "titulocap", "numero": "temporada"}).astype({"cid":int,"sid":int,"temporada":int})
df_capitulos.to_csv("5)capitulos.csv", index=False)

In [273]:
lista_interes = ["sid", "genero"] #Genero de la serie
df_serie_genero = df_serie[lista_interes].drop_duplicates()
#df_serie_genero.to_csv(folder + 'genero_serie.csv', index=True)
df_serie_genero = df_serie_genero.merge(df_gen, on="genero", how="left")
df_serie_genero[df_serie_genero["gid"].isnull()]
df_serie.query("sid == 23")

,sid,cid,serie,titulo,puntuacion,año,clasificacion,genero,numero
27,23.0,132.0,Rick y Morty,Pilot,8.3,2013,PG,NaN,1.0
451,23.0,133.0,Rick y Morty,A Rickle in Time,8.3,2013,PG,NaN,2.0
196,23.0,134.0,Rick y Morty,The Rickshank Rickdemption,8.3,2013,PG,NaN,3.0


In [155]:
items_peliculas = ["pid","titulo","duracion","clasificacion","puntuacion"]
items_genero_peli = ["pid", "genero"]
df_peli = df_multi[~df_multi["pid"].isnull()]
#2 tablas: genero_pelicula y peliculas
df_genero_peli = df_peli[items_genero_peli]
df_peli_final = df_peli[items_peliculas]
df_genero_peli = df_genero_peli.sort_values(by="pid")
#df_genero_peli.to_csv(folder+'genero_pelicula.csv', index=False)
df_genero_peli

,pid,genero
241,1.0,Biografía
359,1.0,Comedia
473,1.0,Drama
141,2.0,Comedia
311,3.0,Drama
...,...,...
552,99.0,Crimen
532,100.0,Comedia
169,100.0,Drama
574,101.0,Horror


In [131]:
df_subgen # Queda igual
df_subgen.to_csv(folder + 'genero_subgenero.csv', index=False)

In [210]:
df_pagos
lista_atributos = ["pago_id", "uid", "pid", "pro_id", "fecha"] #no es necesario el monto
lista_atributos_2 = ["pago_id", "uid", "subs_id", "fecha"]
df_registro_peli_paga = df_pagos[~df_pagos["pid"].isnull()][lista_atributos] #Pago peli pagas
df_pago_subs = df_pagos[df_pagos["pid"].isnull()][lista_atributos_2] # pago subscripción
df_registro_peli_paga = df_registro_peli_paga.sort_values(by="pago_id")
df_pago_subs = df_pago_subs.sort_values(by="pago_id")
#df_pago_subs.to_csv(folder+"pago_subscripcion.csv", index=False)
#df_registro_peli_paga.to_csv(folder+".csv", index=False)
#df_registro_peli_paga
df_pago_subs
#df_registro_peli_paga

,pago_id,uid,subs_id,fecha
3787,6368,29,1.0,2021-06-08
3814,6369,29,1.0,2021-07-08
3356,6370,29,1.0,2021-08-08
3514,6371,29,1.0,2021-09-08
1965,6372,48,2.0,2020-09-27
...,...,...,...,...
3911,10361,90,419.0,2021-08-12
2428,10362,90,419.0,2021-09-12
1832,10363,90,419.0,2021-10-12
2331,10364,90,419.0,2021-11-12


In [212]:
lista = ["id","estado","fecha_inicio","pro_id","uid", "fecha_termino"] # id -> id_subscripcion
df_sub = df_subs[lista]
#df_sub.to_csv(folder+"14)subscripcion.csv", index=False)
df_sub

,id,estado,fecha_inicio,pro_id,uid,fecha_termino
0,352,activa,2020-06-20,7,119,NaN
1,340,activa,2020-08-20,20,5,NaN
2,170,activa,2020-06-02,9,14,NaN
3,245,activa,2021-05-31,9,50,NaN
4,324,activa,2020-08-25,10,63,NaN
...,...,...,...,...,...,...
414,282,cancelada,2020-05-27,15,46,2021-01-01
415,195,cancelada,2020-05-20,11,43,2021-06-08
416,271,activa,2021-03-29,16,23,NaN
417,339,cancelada,2021-11-09,7,5,2021-12-03


In [206]:
df_vis_peli = df_vis[~df_vis["pid"].isnull()][["uid", "pid", "fecha"]]
df_vis_serie = df_vis[~df_vis["cid"].isnull()][["uid","cid","fecha"]]
df_vis_peli.to_csv(folder+"16)vepelicula.csv", index=False)
df_vis_serie.to_csv(folder+"17)veserie.csv", index=False)

In [242]:
df_usuarios = df_usuarios.rename(columns={"id":"uid"}).sort_values(by="uid")
#df_usuarios.to_csv("7)usuarios.csv", index=False)